# 🧾 View as a summary

In [ ]:
#| default_exp repr_str

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
# |hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#| hide
#| export
from typing import Union, Optional as O
from collections import defaultdict
from fastcore.foundation import store_attr
import warnings
import numpy as np

from lovely_numpy.utils import pretty_str, sparse_join, np_to_str_common, in_debugger
from lovely_numpy.utils.config import get_config, set_config, config

In [ ]:
# |hide
np.random.seed(0)
randoms = np.random.randn(100)

In [ ]:
# |exporti
dtnames =   {   "float16": "f16",
                "float32": "f32",
                "float64": "", # Default dtype in numpy
                "uint8": "u8",
                "uint16": "u16",
                "uint32": "u32",
                "uint64": "u64",
                "int8": "i8",
                "int16": "i16",
                "int32": "i32",
                "int64": "i64",
            }

def short_dtype(x: Union[np.ndarray, np.generic]):
    return dtnames.get(x.dtype.name, x.dtype)

In [ ]:
# |hide
test_eq(short_dtype(np.array(1., dtype=np.float16)), "f16")

## Pretty printing

In [ ]:
# |exporti
def plain_repr(x):
    with config(repr=None):
        return repr(x)

In [ ]:
# |export

def lovely( x       :Union[np.ndarray, np.generic], # The data you want to explore 
            plain   :bool   =False,                 # Plain old way
            verbose :bool   =False,                 # Both summaty and plain
            depth   :int    =0,                     # Show deeper summary, up to `depth`
            lvl     :int    =0,                     # Indentation level 
            color   :O[bool]=None                   # Override `get_config().color`
            ) -> str:                               # The summary

    "Pretty-print the stats of a numpy array or scalar"

    if plain or not isinstance(x, (np.ndarray, np.generic)) or np.iscomplexobj(x):
        return plain_repr(x)

    conf = get_config()

    if isinstance(x, np.generic):
        tname = None
    else:
        tname = "array" if type(x) == np.ndarray else type(x).__name__.split(".")[-1]

    shape = str(list(x.shape)) if x.ndim else None
    type_str = sparse_join([tname, shape], sep="")

    color = get_config().color if color is None else color
    if in_debugger(): color = False
    common = np_to_str_common(x, color=color)
    dtype = short_dtype(x)
    
    vals = pretty_str(x) if 0 < x.size <= 10 else None
    res = sparse_join([type_str, dtype, common, vals])

    if verbose:
        res += "\n" + plain_repr(x)

    if depth and x.ndim > 1:
        deep_width = min(x.shape[0], conf.deeper_width) # Print at most this many lines
        deep_lines = [ " "*conf.indent*(lvl+1) + lovely(x[i,:], depth=depth-1, lvl=lvl+1)
                            for i in range(deep_width)]

        # If we were limited by width, print ...
        if deep_width < x.shape[0]: deep_lines.append(" "*conf.indent*(lvl+1) + "...")

        res += "\n" + "\n".join(deep_lines)

        # res += "\n" + "\n".join([
        #     " " * get_config().indent * (lvl+1) +
        #     str(lovely(x[i,:], depth=depth-1, lvl=lvl+1))
        #     for i in range(x.shape[0])])

    return res

In [ ]:
show_doc(lovely)

---

[source](https://github.com/xl0/lovely-numpy/blob/master/lovely_numpy/repr_str.py#L39){target="_blank" style="float:right; font-size:smaller"}

### lovely

>      lovely (x:Union[numpy.ndarray,numpy.generic], plain:bool=False,
>              verbose:bool=False, depth:int=0, lvl:int=0,
>              color:Optional[bool]=None)

Pretty-print the stats of a numpy array or scalar

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| x | Union |  | The data you want to explore |
| plain | bool | False | Plain old way |
| verbose | bool | False | Both summaty and plain |
| depth | int | 0 | Show deeper summary, up to `depth` |
| lvl | int | 0 | Indentation level |
| color | Optional | None | Override `get_config().color` |
| **Returns** | **str** |  | **The summary** |

### Examples 

In [ ]:
nasties = randoms[:12].copy()

nasties[0] *= 10000
nasties[1] /= 10000
nasties[3] = float('inf')
nasties[4] = float('-inf')
nasties[5] = float('nan')
nasties = nasties.reshape((2,6))

In [ ]:
print(lovely(nasties))

array[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!


In [ ]:
print(lovely(randoms[0]))
print(lovely(randoms[:2]))
print(lovely(randoms[:6].reshape(2, 3))) # More than 2 elements -> show statistics
print(lovely(randoms[:11])) # More than 10 -> don't show values


1.764
array[2] μ=1.082 σ=0.682 [1.764, 0.400]
array[2, 3] n=6 x∈[-0.977, 2.241] μ=1.046 σ=1.090 [[1.764, 0.400, 0.979], [2.241, 1.868, -0.977]]
array[11] x∈[-0.977, 2.241] μ=0.684 σ=0.938


In [ ]:
# |hide
test_eq(str(lovely(randoms[0])), "1.764")
test_eq(str(lovely(randoms[:2])), "array[2] μ=1.082 σ=0.682 [1.764, 0.400]")
test_eq(str(lovely(randoms[:6].reshape(2, 3))), "array[2, 3] n=6 x∈[-0.977, 2.241] μ=1.046 σ=1.090 [[1.764, 0.400, 0.979], [2.241, 1.868, -0.977]]")
test_eq(str(lovely(randoms[:11])), "array[11] x∈[-0.977, 2.241] μ=0.684 σ=0.938")

Do we have __any__ floating point nasties? Are the values __all__ zeros?

In [ ]:
# Statistics and range are calculated on good values only, if there are at lest 3 of them.
print(lovely(nasties))

array[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!


In [ ]:
# |hide
test_eq(str(lovely(nasties)),
        'array[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 \x1b[31m+Inf!\x1b[0m \x1b[31m-Inf!\x1b[0m \x1b[31mNaN!\x1b[0m')

In [ ]:
print(lovely(nasties, color=False))

array[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!


In [ ]:
print(lovely(np.array([float("nan")]*11)))

array[11] NaN!


In [ ]:
# |hide
test_eq(str(lovely(np.array([float("nan")]*11))),
        'array[11] \x1b[31mNaN!\x1b[0m')

In [ ]:
print(lovely(np.zeros(12, dtype=np.float16)))
print(lovely(np.array([], dtype=int)))

array[12] f16 all_zeros
array[0] i64 empty


In [ ]:
# |hide
test_eq(str(lovely(np.zeros(12, dtype=np.float16))), 'array[12] f16 \x1b[38;2;127;127;127mall_zeros\x1b[0m')
test_eq(str(lovely(np.array([], dtype=int))), 'array[0] i64 \x1b[38;2;127;127;127mempty\x1b[0m')

In [ ]:
str(lovely(np.array([], dtype=int)))

'array[0] i64 \x1bempty\x1b'

In [ ]:
np.set_printoptions(precision=3)
print(lovely(nasties, verbose=True))

array[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!
array([[ 1.764e+04,  4.002e-05,  9.787e-01,        inf,       -inf,
               nan],
       [ 9.501e-01, -1.514e-01, -1.032e-01,  4.106e-01,  1.440e-01,
         1.454e+00]])


In [ ]:
print(lovely(nasties, plain=True))

array([[ 1.764e+04,  4.002e-05,  9.787e-01,        inf,       -inf,
               nan],
       [ 9.501e-01, -1.514e-01, -1.032e-01,  4.106e-01,  1.440e-01,
         1.454e+00]])


In [ ]:
image = np.load("mysteryman.npy")
image[1,100,100] = float('nan')

print(lovely(image, depth=1))

array[3, 196, 196] f32 n=115248 x∈[-2.118, 2.640] μ=-0.388 σ=1.073 NaN!
  array[196, 196] f32 n=38416 x∈[-2.118, 2.249] μ=-0.324 σ=1.036
  array[196, 196] f32 n=38416 x∈[-1.966, 2.429] μ=-0.274 σ=0.973 NaN!
  array[196, 196] f32 n=38416 x∈[-1.804, 2.640] μ=-0.567 σ=1.178


In [ ]:
# We don't really supposed complex numbers yet
c = np.random.randn(2) + 1j*np.random.randn(2)
print(lovely(c))

array([ 1.883-1.27j , -1.348+0.969j])


In [ ]:
i = np.array([1, 2, 3])
test_eq(str(lovely(i)), "array[3] i64 x∈[1, 3] μ=2.000 σ=0.816 [1, 2, 3]")

In [ ]:
set_config(repr=lovely, str=None)

In [ ]:
np.array([1,2,3])

array[3] i64 x∈[1, 3] μ=2.000 σ=0.816 [1, 2, 3]

In [ ]:
# |hide
test_eq(repr(np.array([1,2,3])),
        'array[3] i64 x∈[1, 3] μ=2.000 σ=0.816 [1, 2, 3]')

In [ ]:
# |hide
with config(repr=None):
    test_eq(repr(np.array([1,2,3])), 'array([1, 2, 3])')
    print(repr(np.array([1, 2, 3])))

array([1, 2, 3])


In [ ]:
# |hide
test_eq(repr(np.array([1,2,3])),
        'array[3] i64 x∈[1, 3] μ=2.000 σ=0.816 [1, 2, 3]')

np.array([1,2,3])

array[3] i64 x∈[1, 3] μ=2.000 σ=0.816 [1, 2, 3]